In [4]:
import csv
import os.path
import configparser as ConfigParser
import pandas as pd
import numpy as np
import requests
import csv
import urllib.parse

# Read Files

In [5]:
# These should be the full file paths or at least relative to the cwd
file_cfg = ConfigParser.ConfigParser()
file_cfg.read('C:\\Users\\asd\\python-workspace\\zucchini\\config\\zucchini.ini')

['C:\\Users\\asd\\python-workspace\\zucchini\\config\\zucchini.ini']

In [6]:
main_dir = os.path.dirname(os.getcwd())
param_file = os.path.join(main_dir, file_cfg.get("Input", "input_file"))
param_file

'C:\\Users\\asd\\python-workspace\\zucchini\\input\\PANGAEAParameterComplete.tab'

In [7]:
main_dir

'C:\\Users\\asd\\python-workspace\\zucchini'

In [8]:
mapping_file = os.path.join(main_dir, file_cfg.get("Input", "mapping_file"))

In [9]:
cols_params = ['Parameter', 'Abbreviation','Unit','ID parameter']

In [10]:
df_mapping = pd.read_csv(mapping_file, header=None, delimiter="\t", encoding='ISO-8859-1',names = ["NativeUOM", "UCUM"])
df_mapping.head()

,NativeUOM,UCUM
0,#,#
1,# of ions,#{ions}
2,#/#,#/#
3,#/#/day,#/#/d
4,#/#/h,#/#/h


In [11]:
df_mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 2 columns):
NativeUOM    1247 non-null object
UCUM         1247 non-null object
dtypes: object(2)
memory usage: 19.6+ KB


In [12]:
df_mapping.nunique()

NativeUOM    1247
UCUM         1206
dtype: int64

In [13]:
pd.concat(g for _, g in df_mapping.groupby("UCUM") if len(g) > 1).head()

,NativeUOM,UCUM
52,#/mm**2,#/mm2
53,#/mm*2,#/mm2
123,10**-6 A m**2/kg,(10^-6.A).m2/kg
125,10**-6 A*m**2/kg,(10^-6.A).m2/kg
621,mA m**2/kg,(mA.m2)/kg


In [14]:
df = pd.read_csv(param_file, header=0, usecols=cols_params, delimiter="\t", encoding='utf-8')

In [15]:
df.head()

,Parameter,Abbreviation,Unit,ID parameter
0,-,NaN,NaN,16431
1,-,NaN,NaN,16432
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m**3,53014
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280
4,"10-Methyldodecanoic acid, particulate",10-methyllauric acid part,nmol/l,30245


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158025 entries, 0 to 158024
Data columns (total 4 columns):
Parameter       158025 non-null object
Abbreviation    158022 non-null object
Unit            122269 non-null object
ID parameter    158025 non-null int64
dtypes: int64(1), object(3)
memory usage: 4.8+ MB


In [17]:
#remove lines with no parameter
df = df[df.Parameter != '-']

In [18]:
df.head(2)

,Parameter,Abbreviation,Unit,ID parameter
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m**3,53014
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280


In [19]:
'nan' in df.Unit.values

False

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158023 entries, 2 to 158024
Data columns (total 4 columns):
Parameter       158023 non-null object
Abbreviation    158022 non-null object
Unit            122269 non-null object
ID parameter    158023 non-null int64
dtypes: int64(1), object(3)
memory usage: 6.0+ MB


In [21]:
#List unique values each cols
df.nunique()

Parameter       127375
Abbreviation    119256
Unit              1337
ID parameter    158023
dtype: int64

In [22]:
df.Unit.unique()[:20]

array(['pg/m**3', 'pg/l', 'nmol/l', 'µg/g', 'per mil PDB', '%', 'pptv',
       'ng/g', 'pmol/l', 'ng/l', 'pg/g', nan, 'ppbv', 'pg/ml', 'nmol/kg',
       'ng/NT', 'per mil', 'µg C/l/day', 'mg C/kg/day', 'nmol/l/day'], dtype=object)

In [23]:
df.Unit.isnull().values.any()

True

In [24]:
list_of_uom= df.Unit.dropna().unique()

In [25]:
df["QuantityKind"] = np.nan
df.head(2)

,Parameter,Abbreviation,Unit,ID parameter,QuantityKind
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m**3,53014,NaN
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280,NaN


# Clean UoM based on the mapping file

In [26]:
matched_uom = []
def validate_units(uom):
    valid_uom = np.nan
    new_uom = df_mapping.loc[df_mapping['NativeUOM'] == uom, 'UCUM']
    if not new_uom.empty:
        valid_uom = new_uom.iloc[0]
        matched_uom.append(uom)
    return valid_uom

In [27]:
for uom_ori in list_of_uom:
    uom_ucum = validate_units(uom_ori)
    if uom_ucum is not None:
        df.loc[df.Unit == uom_ori, 'Unit'] = uom_ucum

In [28]:
total_exist_mapping = df_mapping.NativeUOM.nunique()
print('Total robert mappings: ',total_exist_mapping)
print('Total matched units: ',len(matched_uom), len(set(matched_uom)))

Total robert mappings:  1247
Total matched units:  1226 1226


In [29]:
print('unused uom mappings:')
list(set(df_mapping.NativeUOM.unique()) - set(matched_uom))

unused uom mappings:


['(g C)/(g Chl a)/E',
 'µC/(µE/m**2/s)/mg Chl/h',
 'years',
 'ppm',
 'µmol/g/d',
 'ppt',
 'fmol/g',
 'mg/(µE/m**2/s)/mg Chl/h',
 '#/m**/d',
 'µg/m**/day',
 'gpm',
 'deg s',
 'mg/mg Chl a/h',
 'µmol/m**2/d',
 'nl C/dm**3/day',
 'Ohmm',
 '(mg/mg/h)/(µmol/m**2/s)',
 'mBq/cm**2',
 'per mil CTD',
 'µE/m**2',
 'µE/m**2/s']

In [44]:
df.head(10)

,Parameter,Abbreviation,Unit,ID parameter,QuantityKind
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m3,53014,Density
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280,Density
4,"10-Methyldodecanoic acid, particulate",10-methyllauric acid part,nmol/l,30245,"Amount of Substance Per Unit Volume,Molar Conc..."
5,10-methyl-Hexadecanoic acid,10Me-C16:0,ug/g,46516,Dimensionless
6,"10-methyl-Hexadecanoic acid, d13C",10Me-C16:0 d13C,[per_mil]{PDB},46530,Dimensionless
7,10-methyl-Hexadecanoic acid of total fatty acids,10me16:0,%,97842,Dimensionless
8,10-methyl-Octadecanoic acid of total fatty acids,10me18:0,%,97853,Dimensionless
9,10-methyl-pentadecanoic acid of total fatty acids,10me15:0,%,97840,Dimensionless
10,10-methyl-tetradecanoic acid of total fatty acids,10me14:0,%,97837,Dimensionless
11,"11,12-methylene-Hexadecanoic acid","cyC17:0w5,6",ug/g,46518,Dimensionless


# Update Quantity Kind

In [31]:
uom_not_found =[]
uom_empty_result =[]
quantity_null=[]
list_of_uom_updated= df.Unit.dropna().unique()
for u in list_of_uom_updated:
    q = 'http://dataportals.pangaea.de/test/ucum/?ucum='+ str(u)
    resp = requests.get(q)
    if '"input":' in resp.text:
        data = resp.json()
        if 'error:' in resp.text:
            uom_not_found.append(u)
        else:
            if 'Quantities' in data:
                quantity = data['Quantities']
            else:
                quantity = data['quantities']
            if(quantity):
                quantity_str = ','.join(quantity)
                df.loc[df.Unit == u, 'QuantityKind'] = quantity_str
            else:
                quantity_null.append(u)
    else:
        uom_empty_result.append(u)

In [32]:
len(uom_not_found),len(set(uom_not_found)),uom_not_found[:6]

(131, 131, ['[pptv]', 'pmol/#', 'mg/#/h', 'J/#/h', '10^6.#/g', '10^6.#/m2/d'])

In [33]:
len(uom_empty_result),uom_empty_result[:6]

(73, ['#/cm3', '#', '#/g', '#/(10.cm3)', '#/cm2/a', '#/(50.cm3)'])

In [34]:
len(quantity_null), quantity_null[:6]

(180,
 ['ug{C}/l/d',
  'ng/l/d',
  '10^6.[ru]/g',
  '[dpm]/cm2/ka',
  '[meq]/kg',
  '[dpm]/m3'])

In [35]:
df.head(5)

,Parameter,Abbreviation,Unit,ID parameter,QuantityKind
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m3,53014,Density
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280,Density
4,"10-Methyldodecanoic acid, particulate",10-methyllauric acid part,nmol/l,30245,"Amount of Substance Per Unit Volume,Molar Conc..."
5,10-methyl-Hexadecanoic acid,10Me-C16:0,ug/g,46516,Dimensionless
6,"10-methyl-Hexadecanoic acid, d13C",10Me-C16:0 d13C,[per_mil]{PDB},46530,Dimensionless


In [36]:
df.count()

Parameter       158023
Abbreviation    158022
Unit            122073
ID parameter    158023
QuantityKind     63313
dtype: int64

In [37]:
len(df.QuantityKind.dropna().unique())

68

In [38]:
def write_data_tofile(datalist, filename):
    with open(filename,'w') as outfile:
        for entries in datalist:
            outfile.write(entries)
            outfile.write("\n")

In [39]:
write_data_tofile(uom_not_found,os.path.join(main_dir, 'output\\uom_not_found.csv'))
write_data_tofile(uom_empty_result,os.path.join(main_dir, 'output\\uom_empty_result.csv'))
write_data_tofile(quantity_null,os.path.join(main_dir, 'output\\quantity_null.csv'))

# Validate Units with UCUM Service

In [40]:
ucum_compliant_uom =[]
ucum_non_compliant_uom =[]
ucum_rh = df_mapping.UCUM.dropna().unique()
for u in ucum_rh:
    #quoting special characters and appropriately encoding non-ASCII text
    unit_encoded = urllib.parse.quote(u)
    q = 'https://ucum.nlm.nih.gov/ucum-service/v1/isValidUCUM/'+ unit_encoded
    resp = requests.get(q)
    if resp.status_code == 200 and resp.text == 'true':
        ucum_compliant_uom.append(u)
    else:
        ucum_non_compliant_uom.append(u)

In [41]:
print(len(set(ucum_compliant_uom)),len(ucum_compliant_uom))

914 914


In [42]:
print(len(set(ucum_non_compliant_uom)),len(ucum_non_compliant_uom))

292 292


In [43]:
ucum_non_compliant_uom[:10]

['#',
 '#{ions}',
 '#/#',
 '#/#/d',
 '#/#/h',
 '#/#/s',
 '#/(10.cm2)',
 '#/(10.cm3)',
 '#/(10.g)/ka',
 '#/(10.km)']

# Elastic Query